In [2]:
import datetime
from datetime import date
import requests
import pprint
from bs4 import BeautifulSoup
import pandas as pd
import re
import pickle
from urllib.request import urlopen

#AWS Test

In [5]:
URL = 'https://fbref.com/en/comps/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
# Big 5 European Leagues (Spain, England, Germany, France, Italy)

big_5_leagues = []

for j in soup.find_all('tbody')[2].find_all("tr", {"class": "gender-m"}):
    if (j.find('td') != None):
        big_5_leagues.append(j.find('a')['href'])

big_5_leagues = big_5_leagues[:-1]

big_5_leagues

['/en/comps/9/history/Premier-League-Seasons',
 '/en/comps/12/history/La-Liga-Seasons',
 '/en/comps/13/history/Ligue-1-Seasons',
 '/en/comps/20/history/Bundesliga-Seasons',
 '/en/comps/11/history/Serie-A-Seasons']

In [7]:
# function to obtain league/season URLs

def get_all_seasons(url):
    URL = 'https://fbref.com/' + url
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    url_list = []
    
    for row in soup.find_all('tr'):
        if row.find('th',{"scope":"row"}) != None:
            url_list.append((row.find('a')['href']))
    
    return url_list

In [8]:
# All Seasons Big 5 Leagues

all_seasons_big_5 = []

for i in big_5_leagues:
    league_seasons = get_all_seasons(i)
    all_seasons_big_5 += league_seasons

all_seasons_big_5

['/en/comps/9/Premier-League-Stats',
 '/en/comps/9/10728/2020-2021-Premier-League-Stats',
 '/en/comps/9/3232/2019-2020-Premier-League-Stats',
 '/en/comps/9/1889/2018-2019-Premier-League-Stats',
 '/en/comps/9/1631/2017-2018-Premier-League-Stats',
 '/en/comps/9/1526/2016-2017-Premier-League-Stats',
 '/en/comps/9/1467/2015-2016-Premier-League-Stats',
 '/en/comps/9/733/2014-2015-Premier-League-Stats',
 '/en/comps/9/669/2013-2014-Premier-League-Stats',
 '/en/comps/9/602/2012-2013-Premier-League-Stats',
 '/en/comps/9/534/2011-2012-Premier-League-Stats',
 '/en/comps/9/467/2010-2011-Premier-League-Stats',
 '/en/comps/9/400/2009-2010-Premier-League-Stats',
 '/en/comps/9/338/2008-2009-Premier-League-Stats',
 '/en/comps/9/282/2007-2008-Premier-League-Stats',
 '/en/comps/9/229/2006-2007-Premier-League-Stats',
 '/en/comps/9/183/2005-2006-Premier-League-Stats',
 '/en/comps/9/146/2004-2005-Premier-League-Stats',
 '/en/comps/9/112/2003-2004-Premier-League-Stats',
 '/en/comps/9/84/2002-2003-Premier-Lea

In [9]:
# print(len(all_seasons))
print(len(all_seasons_big_5))

159


In [10]:
def get_all_player_urls(url_list):
    
    final_player_urls = []

    for i in url_list:
        final_pos = i.rindex('/')

        final_player_urls.append(i[:final_pos+1]+'stats/'+i[final_pos+1:])

    return final_player_urls

In [11]:
# Getting final_stats for all players in the Big 5 Leagues

final_stats_big_5 = get_all_player_urls(all_seasons_big_5)

In [12]:
final_stats_big_5

['/en/comps/9/stats/Premier-League-Stats',
 '/en/comps/9/10728/stats/2020-2021-Premier-League-Stats',
 '/en/comps/9/3232/stats/2019-2020-Premier-League-Stats',
 '/en/comps/9/1889/stats/2018-2019-Premier-League-Stats',
 '/en/comps/9/1631/stats/2017-2018-Premier-League-Stats',
 '/en/comps/9/1526/stats/2016-2017-Premier-League-Stats',
 '/en/comps/9/1467/stats/2015-2016-Premier-League-Stats',
 '/en/comps/9/733/stats/2014-2015-Premier-League-Stats',
 '/en/comps/9/669/stats/2013-2014-Premier-League-Stats',
 '/en/comps/9/602/stats/2012-2013-Premier-League-Stats',
 '/en/comps/9/534/stats/2011-2012-Premier-League-Stats',
 '/en/comps/9/467/stats/2010-2011-Premier-League-Stats',
 '/en/comps/9/400/stats/2009-2010-Premier-League-Stats',
 '/en/comps/9/338/stats/2008-2009-Premier-League-Stats',
 '/en/comps/9/282/stats/2007-2008-Premier-League-Stats',
 '/en/comps/9/229/stats/2006-2007-Premier-League-Stats',
 '/en/comps/9/183/stats/2005-2006-Premier-League-Stats',
 '/en/comps/9/146/stats/2004-2005-Prem

In [13]:
def get_player_table(url):
    url = 'https://fbref.com/' + url
    res = requests.get(url)
    ## The next two lines get around the issue with comments breaking the parsing.
    comm = re.compile("<!--|-->")
    soup = BeautifulSoup(comm.sub("",res.text),'lxml')
    all_tables = soup.findAll("tbody")
    player_table = all_tables[2]

    return player_table

In [14]:
def get_player_urls(url_list):
    
    player_url_list = []

    for url in url_list:
        url = 'https://fbref.com/' + url
        res = requests.get(url)
        ## The next two lines get around the issue with comments breaking the parsing.
        comm = re.compile("<!--|-->")
        soup = BeautifulSoup(comm.sub("",res.text),'lxml')
        all_tables = soup.findAll("tbody")
        try:
            player_table = all_tables[2]
        except:
            next

        for row in player_table.find_all('a'):
            if "matchlogs" in row['href']:
                player_url_list.append(row['href'])

    return player_url_list

In [15]:
stats = ["player","nationality","position","squad","age","birth_year","games","games_starts","minutes","goals","assists","pens_made","pens_att","cards_yellow","cards_red","goals_per90","assists_per90","goals_assists_per90","goals_pens_per90","goals_assists_pens_per90","xg","npxg","xa","xg_per90","xa_per90","xg_xa_per90","npxg_per90","npxg_xa_per90"]

In [16]:
def get_frame(features, player_table):
    pre_df_player = dict()
    features_wanted_player = features
    rows_player = player_table.find_all('tr')
    for row in rows_player:
        if(row.find('th',{"scope":"row"}) != None):
    
            for f in features_wanted_player:
                cell = row.find("td",{"data-stat": f})
                a = cell.text.strip().encode()
                text=a.decode("utf-8")
                if(text == ''):
                    text = '0'
                if((f!='player')&(f!='nationality')&(f!='position')&(f!='squad')&(f!='age')&(f!='birth_year')):
                    text = float(text.replace(',',''))
                if f in pre_df_player:
                    pre_df_player[f].append(text)
                else:
                    pre_df_player[f] = [text]
    df_player = pd.DataFrame.from_dict(pre_df_player)

    return df_player

In [17]:
#Getting player tables for big 5 leagues

player_table_big_5 = get_player_urls(final_stats_big_5)

len(player_table_big_5)

81273

In [5]:
# # Saving player_table as pickle file
# import pickle

# with open('/Volumes/GoogleDrive/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/Pickle Files/player_table.pickle', 'wb') as handle:
#     pickle.dump(player_table, handle, protocol=pickle.HIGHEST_PROTOCOL)


# pd.DataFrame(player_table_big_5).to_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/player_table_big_5.csv')

player_table_big_5 = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV Files/player_table_big_5.csv')

In [12]:
player_table_big_5 = list(player_table_big_5['0'])

In [13]:
# Pull all player stats for all competitions

def get_players_all_competitions(player_list):
    
    player_urls = []

    for i in player_list:
        player_urls.append('https://fbref.com/en/players/' + i.split('/')[3:4][0] + '/all_comps/' 
                            + i.split('/')[7:][0].replace("-Match-Logs", "") + '/-Stats---All-Competitions')

    return list(set(player_urls))

player_all_competitions = get_players_all_competitions(player_table_big_5)
len(player_all_competitions)


19569

In [25]:
# player_all_competitions

# substring_in_list = any('dea698d9' in string for string in player_all_competitions)
                        
# substring_in_list       


True

In [14]:
# Generate the match log urls for all players across all 

def get_player_match_logs(player_list_summary, line):
    
    res = requests.get(player_list_summary[line])
    soup = BeautifulSoup(res.text,'lxml')

    match_logs_list = []

    for i in soup.find_all('tbody'):
        for j in i.find_all('td', {'data-stat':'matches'}):
            if j.find('a') != None:
                if 'summary' in j.find('a')['href']:
                    match_logs_list.append(j.find('a')['href'])
                    
    return list(set(match_logs_list))

In [15]:
player_all_competitions_df = pd.DataFrame(player_all_competitions)

player_all_competitions_df['FBRefID'] = player_all_competitions_df[0].str.split('/').str[5]

# player_all_competitions_df[player_all_competitions_df['FBRefID'] == 'dea698d9']

In [16]:
fbref_to_tm_mapping = pd.read_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/fbref_to_tm_mapping.csv', encoding='latin-1')
fbref_to_tm_mapping['FBRefID'] = fbref_to_tm_mapping['UrlFBref'].str.split('/').str[5]
fbref_to_tm_mapping['TMID'] = fbref_to_tm_mapping['UrlTmarkt'].str.split('/').str[6]

In [17]:
player_all_competitions_filtered = player_all_competitions_df.merge(fbref_to_tm_mapping, left_on='FBRefID', right_on='FBRefID', how='inner')


In [18]:
player_all_competitions_filtered[player_all_competitions_filtered['FBRefID'] == 'dea698d9']


player_all_competitions_filtered_list = list(player_all_competitions_filtered[0])

In [19]:
len(player_all_competitions_filtered)

5192

In [20]:
player_all_competitions_filtered_list

['https://fbref.com/en/players/c2846353/all_comps/Marc-Navarro/-Stats---All-Competitions',
 'https://fbref.com/en/players/98ea5115/all_comps/David-Raya/-Stats---All-Competitions',
 'https://fbref.com/en/players/b18a3023/all_comps/Junior-Alonso/-Stats---All-Competitions',
 'https://fbref.com/en/players/22721b88/all_comps/Jordan-Amavi/-Stats---All-Competitions',
 'https://fbref.com/en/players/b581987a/all_comps/Andy-Carroll/-Stats---All-Competitions',
 'https://fbref.com/en/players/a6138e4a/all_comps/Jessy-Deminguet/-Stats---All-Competitions',
 'https://fbref.com/en/players/68796d61/all_comps/Shinji-Okazaki/-Stats---All-Competitions',
 'https://fbref.com/en/players/c2dfedeb/all_comps/Filip-Krovinovic/-Stats---All-Competitions',
 'https://fbref.com/en/players/3d50bcdb/all_comps/Moise-Kean/-Stats---All-Competitions',
 'https://fbref.com/en/players/c8d7131c/all_comps/Ridgeciano-Haps/-Stats---All-Competitions',
 'https://fbref.com/en/players/8d03ca5a/all_comps/Jamie-Shackleton/-Stats---All-C

In [21]:

# Total length of player_all_competitions is 5192


match_logs_list = []

# 1st batch 0:5000 - DONE
count = 0
for i in range(len(player_all_competitions_filtered_list)):
    match_logs_list.extend(get_player_match_logs(player_all_competitions_filtered_list, i))
    count += 1
    sys.stdout.write("\r{0} percent".format((count / len(player_all_competitions_filtered_list)*100)))
    sys.stdout.flush()

0.34668721109399075 percentt

In [ ]:
# export match_logs_list_urls

pd.DataFrame(match_logs_list).to_csv('G:/.shortcut-targets-by-id/1KUGn_35OjAoOP2puz6yG-2g_8LBxvDG_/SIADS 697 - Capstone/CSV files/match_logs_list_urls.csv')